In [2]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

from sklearn.datasets import load_digits
digits = load_digits()

In [2]:
from random import randint

for i in range(64):
    ax = plt.subplot(8, 8, i+1)
    ax.axis('off')
    plt.imshow(digits.images[randint(0, digits.images.shape[0])], cmap='Greys')
plt.show()

In [3]:
X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[:1617]
y = digits.target[:1617]

In [4]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [6]:
def get_winner(class1):
    
    numbers = [*range(10)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1617
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, 375)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-mnist2")
    [stats, winner] = run(config_path)
    
    return winner

In [7]:
#class1 = [0, 2, 4, 6, 8]
#winner1 = get_winner(class1)

In [8]:
winner_list = []
classes = [[8, 9],[4,5,6,7],[2,3,6,7],[1,3,5,7,9]]

for class1 in classes:
    winner = get_winner(class1)
    winner_list.append(winner)

number of class1:319
number of class2:1298

 ****** Running generation 0 ****** 

Population's average fitness: 0.55300 stdev: 0.21486
Best fitness: 0.80581 - size: (2, 13) - species 1 - id 149
Average adjusted fitness: 0.399
Mean genetic distance 2.908, standard deviation 0.501
Population of 200 members in 20 species
Total extinctions: 0
Generation time: 5.223 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.69239 stdev: 0.16242
Best fitness: 0.80767 - size: (2, 14) - species 8 - id 283
Average adjusted fitness: 0.460
Mean genetic distance 2.990, standard deviation 0.476
Population of 200 members in 27 species
Total extinctions: 0
Generation time: 5.397 sec (5.310 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.70544 stdev: 0.15474
Best fitness: 0.81694 - size: (3, 15) - species 1 - id 532
Average adjusted fitness: 0.488
Mean genetic distance 3.041, standard deviation 0.459
Population of 197 members in 33 species
Total extincti

Population of 195 members in 15 species
Total extinctions: 0
Generation time: 8.825 sec (8.795 average)

 ****** Running generation 48 ****** 

Population's average fitness: 0.75274 stdev: 0.11766
Best fitness: 0.85962 - size: (7, 29) - species 23 - id 6663
Average adjusted fitness: 0.559
Mean genetic distance 3.207, standard deviation 0.572
Population of 205 members in 15 species
Total extinctions: 0
Generation time: 8.853 sec (8.816 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.73027 stdev: 0.15575
Best fitness: 0.85962 - size: (7, 29) - species 23 - id 6663
Average adjusted fitness: 0.521
Mean genetic distance 3.213, standard deviation 0.586
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 9.077 sec (8.881 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.74756 stdev: 0.13366
Best fitness: 0.85962 - size: (7, 29) - species 23 - id 6663
Average adjusted fitness: 0.497
Mean genetic distance 

Mean genetic distance 3.267, standard deviation 0.617
Population of 195 members in 14 species
Total extinctions: 0
Generation time: 10.961 sec (10.561 average)

 ****** Running generation 72 ****** 

Population's average fitness: 0.75466 stdev: 0.14062
Best fitness: 0.86085 - size: (9, 35) - species 23 - id 11001
Average adjusted fitness: 0.557
Mean genetic distance 3.236, standard deviation 0.624
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 10.564 sec (10.600 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.74846 stdev: 0.14104
Best fitness: 0.86085 - size: (9, 35) - species 23 - id 11001
Average adjusted fitness: 0.554
Mean genetic distance 3.245, standard deviation 0.616
Population of 195 members in 15 species
Total extinctions: 0
Generation time: 10.778 sec (10.585 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.74061 stdev: 0.16320
Best fitness: 0.86085 - size: (9, 35) - species 23 - 

Mean genetic distance 3.340, standard deviation 0.678
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 12.953 sec (12.250 average)

 ****** Running generation 96 ****** 

Population's average fitness: 0.75760 stdev: 0.13067
Best fitness: 0.86580 - size: (18, 44) - species 59 - id 15391
Average adjusted fitness: 0.564
Mean genetic distance 3.293, standard deviation 0.687
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 13.567 sec (12.421 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.76219 stdev: 0.12565
Best fitness: 0.86580 - size: (18, 44) - species 59 - id 15391
Average adjusted fitness: 0.563
Mean genetic distance 3.290, standard deviation 0.705
Population of 195 members in 13 species
Total extinctions: 0
Generation time: 15.584 sec (12.801 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.75247 stdev: 0.14453
Best fitness: 0.86580 - size: (18, 44) - species 59

Population's average fitness: 0.76154 stdev: 0.12614
Best fitness: 0.89177 - size: (19, 46) - species 59 - id 19288
Average adjusted fitness: 0.564
Mean genetic distance 3.240, standard deviation 0.605
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 13.630 sec (13.766 average)

 ****** Running generation 120 ****** 

Population's average fitness: 0.76839 stdev: 0.12107
Best fitness: 0.89177 - size: (19, 46) - species 59 - id 19288
Average adjusted fitness: 0.569
Mean genetic distance 3.237, standard deviation 0.603
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 14.063 sec (13.797 average)

 ****** Running generation 121 ****** 

Population's average fitness: 0.76404 stdev: 0.12069
Best fitness: 0.89177 - size: (19, 46) - species 59 - id 19288
Average adjusted fitness: 0.562
Mean genetic distance 3.214, standard deviation 0.611
Population of 201 members in 15 species
Total extinctions: 0
Generation time: 14.209 sec (13.859 ave

...

Population's average fitness: 0.68391 stdev: 0.12449
Best fitness: 0.86827 - size: (10, 34) - species 120 - id 13677
Average adjusted fitness: 0.291
Mean genetic distance 2.990, standard deviation 0.489
Population of 198 members in 18 species
Total extinctions: 0
Generation time: 13.912 sec (13.789 average)

 ****** Running generation 265 ****** 

Population's average fitness: 0.67474 stdev: 0.13406
Best fitness: 0.86827 - size: (10, 34) - species 120 - id 13677
Average adjusted fitness: 0.296
Mean genetic distance 2.981, standard deviation 0.498
Population of 197 members in 19 species
Total extinctions: 0
Generation time: 14.137 sec (13.859 average)

 ****** Running generation 266 ****** 

Population's average fitness: 0.68414 stdev: 0.13612
Best fitness: 0.86827 - size: (10, 34) - species 120 - id 13677
Average adjusted fitness: 0.368
Mean genetic distance 2.992, standard deviation 0.492
Population of 198 members in 19 species
Total extinctions: 0
Generation time: 13.145 sec (13.833 

Mean genetic distance 2.991, standard deviation 0.547
Population of 202 members in 13 species
Total extinctions: 0
Generation time: 11.785 sec (12.212 average)

 ****** Running generation 288 ****** 

Population's average fitness: 0.68536 stdev: 0.13260
Best fitness: 0.87755 - size: (16, 44) - species 118 - id 47991
Average adjusted fitness: 0.274
Mean genetic distance 2.983, standard deviation 0.550
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 11.951 sec (12.166 average)

 ****** Running generation 289 ****** 

Population's average fitness: 0.68483 stdev: 0.13300
Best fitness: 0.87755 - size: (16, 44) - species 118 - id 47991
Average adjusted fitness: 0.342
Mean genetic distance 2.984, standard deviation 0.534
Population of 204 members in 14 species
Total extinctions: 0
Generation time: 12.863 sec (12.168 average)

 ****** Running generation 290 ****** 

Population's average fitness: 0.69229 stdev: 0.12803
Best fitness: 0.87755 - size: (16, 44) - speci

Population's average fitness: 0.66499 stdev: 0.12303
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.272
Mean genetic distance 2.903, standard deviation 0.523
Population of 202 members in 10 species
Total extinctions: 0
Generation time: 15.936 sec (13.035 average)

 ****** Running generation 312 ****** 

Population's average fitness: 0.66628 stdev: 0.12215
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.208
Mean genetic distance 2.885, standard deviation 0.526
Population of 202 members in 10 species
Total extinctions: 0
Generation time: 13.489 sec (13.161 average)

 ****** Running generation 313 ****** 

Population's average fitness: 0.67109 stdev: 0.12124
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.351
Mean genetic distance 2.924, standard deviation 0.529
Population of 203 members in 11 species
Total extinctions: 0
Generation time: 13.236 sec (13.295 

Mean genetic distance 2.899, standard deviation 0.541
Population of 200 members in 10 species
Total extinctions: 0
Generation time: 13.173 sec (13.099 average)

 ****** Running generation 335 ****** 

Population's average fitness: 0.68689 stdev: 0.12126
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.329
Mean genetic distance 2.855, standard deviation 0.570
Population of 202 members in 9 species
Total extinctions: 0
Generation time: 12.941 sec (13.098 average)

 ****** Running generation 336 ****** 

Population's average fitness: 0.67309 stdev: 0.12860
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.377
Mean genetic distance 2.846, standard deviation 0.569
Population of 202 members in 9 species
Total extinctions: 0
Generation time: 15.250 sec (13.360 average)

 ****** Running generation 337 ****** 

Population's average fitness: 0.67371 stdev: 0.12859
Best fitness: 0.88188 - size: (19, 50) - species

Population's average fitness: 0.67947 stdev: 0.12784
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.292
Mean genetic distance 2.937, standard deviation 0.533
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 17.283 sec (16.639 average)

 ****** Running generation 359 ****** 

Population's average fitness: 0.68470 stdev: 0.12954
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.429
Mean genetic distance 2.978, standard deviation 0.528
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 14.712 sec (16.545 average)

 ****** Running generation 360 ****** 

Population's average fitness: 0.69165 stdev: 0.12553
Best fitness: 0.88188 - size: (19, 50) - species 137 - id 51769
Average adjusted fitness: 0.238
Mean genetic distance 2.972, standard deviation 0.550
Population of 198 members in 13 species
Total extinctions: 0
Generation time: 15.700 sec (16.447 

In [3]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-mnist2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [10]:
winner_net0 = neat.nn.FeedForwardNetwork.create(winner_list[0], config)

In [11]:
def get_pred_real(i):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(10)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[1617:]
    test_y = digits.target[1617:]

    list_y = test_y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("class1:{}/{}:class2".format(len(class1_label),len(class2_label)))

    testsamplesize = 180
    test_x_inputs =[tuple(c) for c in test_X[class1_indexes].tolist()] + [tuple(c) for c in test_X[class2_indexes].tolist()]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    acc = np.sum(pred_outputs == real_outputs)/testsamplesize
    print("Accuracy: {}".format(acc))
    
    return[pred_outputs, real_outputs]

In [13]:
pred = []
real = []
for i in range(4):
    [pred_outputs, real_outputs] = get_pred_real(i)
    pred.append(pred_outputs)
    real.append(real_outputs)

class1:35/145:class2
Accuracy: 0.9111111111111111
class1:75/105:class2
Accuracy: 0.9166666666666666
class1:72/108:class2
Accuracy: 0.8555555555555555
class1:92/88:class2
Accuracy: 0.8


In [14]:
def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(10)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[j]
    test_y = digits.target[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

In [15]:
number_codes = [[0,0,0,0],
[0,0,0,1],
[0,0,1,0],
[0,0,1,1],
[0,1,0,0],
[0,1,0,1],
[0,1,1,0],
[0,1,1,1],
[1,0,0,0],
[1,0,0,1]]

In [16]:
pred_value = []
error_list = []

for j in range(1617,1617+180):
    pred = []
    for i in range(4):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(10):
        error.append(15 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

In [20]:
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])

In [21]:
np.sum(list_P == digits.target[1617:])/180

0.6

In [22]:
digits.target[1617:]

array([5, 2, 8, 0, 1, 7, 6, 3, 2, 1, 7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4,
       3, 1, 4, 0, 5, 3, 6, 9, 6, 1, 7, 5, 4, 4, 7, 2, 2, 5, 7, 9, 5, 4,
       4, 9, 0, 8, 9, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
       6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9,
       8, 9, 8, 4, 1, 7, 7, 3, 5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3,
       3, 7, 3, 3, 4, 6, 6, 6, 4, 9, 1, 5, 0, 9, 5, 2, 8, 2, 0, 0, 1, 7,
       6, 3, 2, 1, 7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3,
       6, 9, 6, 1, 7, 5, 4, 4, 7, 2, 8, 2, 2, 5, 7, 9, 5, 4, 8, 8, 4, 9,
       0, 8, 9, 8])

In [27]:
# list_P = []
# for i in pred_value:
#     print(i[0])

In [23]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(list_P, digits.target[1617:])
print(cm)

[[16 13  0  0  0  0  0  1  3  0]
 [ 0  3  0  6  0  2  0  0  3  2]
 [ 0  1 14  1  0  0  0  2  1  0]
 [ 0  1  3  7  0  0  0  3  0  1]
 [ 0  0  0  0 14  1 10  1  1  0]
 [ 0  0  0  0  4 14  0  0  0  0]
 [ 0  0  0  0  0  0  8  2  0  0]
 [ 0  0  0  2  0  1  0 10  0  0]
 [ 0  0  0  0  0  0  0  0  8  1]
 [ 0  1  0  2  2  0  0  0  1 14]]


In [24]:
import seaborn as sn


df_cm = pd.DataFrame(cm, index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.xlabel('Real Label')
plt.ylabel('Prediction Label')
plt.show()

In [25]:
plt.hist(list_P)

(array([33., 16., 19., 15., 27., 18., 10., 13.,  9., 20.]),
 array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ]),
 <a list of 10 Patch objects>)

In [26]:
plt.hist(digits.target[1617:])

(array([16., 19., 17., 18., 20., 18., 18., 19., 17., 18.]),
 array([0. , 0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2, 8.1, 9. ]),
 <a list of 10 Patch objects>)

In [27]:
list_P = []
for i,p_value in enumerate(pred_value):
    print("#{} min_error:{} index:{} real:{}".format(i, error_list[i], p_value[0], digits.target[1617+i]))


#0 min_error:11 index:[5] real:5
#1 min_error:11 index:[2] real:2
#2 min_error:11 index:[1] real:8
#3 min_error:11 index:[0] real:0
#4 min_error:11 index:[0] real:1
#5 min_error:11 index:[2] real:7
#6 min_error:11 index:[4] real:6
#7 min_error:11 index:[3] real:3
#8 min_error:11 index:[2] real:2
#9 min_error:11 index:[1] real:1
#10 min_error:11 index:[6] real:7
#11 min_error:11 index:[9] real:4
#12 min_error:11 index:[6] real:6
#13 min_error:11 index:[3] real:3
#14 min_error:11 index:[1] real:1
#15 min_error:11 index:[1] real:3
#16 min_error:12 index:[5 9] real:9
#17 min_error:11 index:[0] real:1
#18 min_error:11 index:[0] real:7
#19 min_error:11 index:[6] real:6
#20 min_error:11 index:[8] real:8
#21 min_error:11 index:[4] real:4
#22 min_error:11 index:[3] real:3
#23 min_error:11 index:[3] real:1
#24 min_error:11 index:[4] real:4
#25 min_error:11 index:[0] real:0
#26 min_error:11 index:[1] real:5
#27 min_error:11 index:[1] real:3
#28 min_error:11 index:[4] real:6
#29 min_error:11 index